In [ ]:
pip install langgraph langchain-community langchain-anthropic tavily-python typing-extensions langchain-openai gradio


PARTE 1: struttura vera e propria del codice contenente gli import, le definizioni di base e le varie funzioni che permettono l'esecuzione del codice

In [ ]:
#|---------------------------------------IMPORT DELLE LIBRERIE. DEFINIZIONE DELLO STATO. FUNZIONI DI SALVATAGGIO E CARICAMENTO MEMORIA---------------------------------------|
from langgraph.graph import StateGraph, START, END
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from IPython.display import Image, display, Markdown
from typing import List, Dict, Optional
from typing_extensions import TypedDict
from datetime import datetime
import json
import os
import re 
import threading
import gradio as gr
from dotenv import load_dotenv

load_dotenv()

# Definiamo la struttura dello stato condiviso dell'agente sotto forma di TypedDict. Ogni campo rappresenta una parte specifica della "memoria" del blog agent.
class BlogState(TypedDict):
    suggested_topics: List[Dict]
    draft: List[str]
    draft_titles: List[str]
    selected_topic: Optional[str]
    last_research: List[Dict]
    schedule_post: List[Dict]

llm = ChatAnthropic(model="claude-3-haiku-20240307", max_tokens=4096)
tavily_tool = TavilySearchResults(max_results=5)

# Definiamo il percorso predefinito del file in cui salveremo la memoria dell'agente.
STATE_FILE_PATH = "blog_agent_memory.json"

# Funzione che permette di salvare lo stato su disco, in formato JSON. Quesot permette di mantenere la continuità del lavoro tra diverse sessioni o esecuzioni.
# Usiamo indentazione e unicode leggibile per facilitare la lettura del file. 
# In caso di errore (es. permessi, file bloccato...), lo segnaliamo in console.
def save_state_to_disk(state: BlogState, path: str = STATE_FILE_PATH):
    try:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(state, f, indent=2, ensure_ascii=False)
    except Exception as e:
        print(f"Errore nel salvataggio dello stato: {e}")

# Funzione per caricare lo stato precedentemente salvato. 
# Se il file esiste e può essere letto ricostruiamo l'intera struttura dello stato, usando .get() per assicurarci che ogni chiave sia presente (altrimenti usiamo un default vuoto).
# In caso contrario, ritorna uno stato vuoto inizializzato correttamente.
def load_state_from_disk(path: str = STATE_FILE_PATH) -> BlogState:
    if os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
            return {
                
                "suggested_topics": data.get("suggested_topics", []),
                "draft": data.get("draft", []),
                "draft_titles": data.get("draft_titles", []),
                "selected_topic": data.get("selected_topic", []),
                "last_research": data.get("last_research", []),
                "schedule_post": data.get("schedule_post", [])
            }
        except Exception as e:
            print(f"Errore nel caricamento dello stato: {e}")
    return {
        "suggested_topics": [],
        "draft": [],
        "draft_titles": [],
        "selected_topic": [],
        "last_research": [],
        "schedule_post": []
    }


#|---------------------------------------FUNZIONE CHE PERMETTERE DI GENERARE DEI TOPIC IN BASE ALL'INPUT DELL'UTENTE---------------------------------------|


def suggest_topics(state: BlogState, user_input: str):
    # Recuperiamo gli ultimi 10 titoli dei post generati in precedenza, in modo da evitare che il modello riproponga idee troppo simili.
    recent_topics = state.get("draft_titles", [])[-10:]

    # Costruiamo un prompt dettagliato da fornire al modello LLM. Il prompt descrive:
    # - Il compito (generare idee per post di tipo "how-to"),
    # - I requisiti e vincoli qualitativi,
    # - Il formato desiderato della risposta,
    # - E specifica anche che l'output deve essere in italiano.
    prompt = f"""**Task**: Genera **da 3 a 5 idee** per un post blog di tipo **'how-to'** (guida pratica), partendo dal tema: '{user_input}'. 
        Il post deve spiegare **come eseguire un'attività specifica**, dettagliando il processo e includendo suggerimenti utili.
        Il post deve essere chiaro, utile e adatto a un pubblico che sta cercando di **imparare o migliorare una competenza**.

        **Requisiti per l'idea**:
        - Deve includere procedure passo-passo, tecniche, o soluzioni pratiche a problemi comuni.
        - Deve includere argomenti che sono facilmente comprensibili e che possono essere implementati dai lettori con le risorse disponibili.
        - Può suggerire strumenti, materiali o risorse che potrebbero essere utili durante il processo descritto.

        **Vincoli**:
        1. Evitare questi topic recenti: {recent_topics}. Non ripetere argomenti trattati recentemente.
        2. Per ogni suggerimento fornire:
        - un **Titolo accattivante** che stimoli l'interesse del lettore e descriva chiaramente il contenuto della guida.
        - il **Tipo** del post (sempre "how-to" in questo caso).
        - una **Motivazione** (anche più di una frase) che spiega perché il post è utile e come risolverà un problema o semplificherà una attività per il lettore.
        3. Gli argomenti devono essere pratici e applicabili. **Esempi di categorie potrebbero includere**:
        - Tecniche manuali (es. "Come fare la manutenzione di un attrezzo da giardino").
        - Software e strumenti (es. "Come ottimizzare le performance di un sito web").
        - Attività quotidiane o di vita pratica (es. "Come organizzare un armadio in modo efficiente").
        4. Il contenuto deve essere scritto in modo che il lettore possa facilmente seguire le istruzioni **senza conoscenze pregresse**, includendo eventualmente consigli o risorse utili.
        5. Ogni idea deve essere originale e non troppo simile ad altri post già suggeriti.

        **Lingua dell'output**: l'intero risultato deve essere scritto in **italiano**.

        **Formato dell' output**:
        ```json
        [
            {{
                "title": "Titolo del post",
                "type": "how-to",
                "justification": "Descrizione dettagliata dell'argomento e del processo, e perché è utile per i lettori."
            }},
            ...
        ]
        ```
    """

    # Inviamo il prompt al modello linguistico (LLM), che restituirà un contenuto generato in formato JSON.
    response = llm.invoke(prompt)
    try:
        # Alcuni modelli restituiscono il JSON racchiuso dentro blocchi markdown (```json ... ```). 
        # In questo caso, estraiamo solo la parte interna al blocco per poterla decodificare.
        # Decodifichiamo il contenuto JSON in una struttura dati Python (lista di dizionari).
        content = response.content
        if '```json' in content:
            content = content.split('```json')[1].split('```')[0]
        suggestions = json.loads(content)

        # Verifichiamo che ogni suggerimento contenga le tre chiavi obbligatorie: title, type e justification. In caso contrario, solleviamo un errore.
        for topic in suggestions:
            if not all(key in topic for key in ['title', 'type', 'justification']):
                raise ValueError("Formato topic non valido")
            
        display(Markdown(f"🟢 È stato scelto il seguente argomento: {user_input}"))

        display(Markdown("🟢 Topic suggeriti:"))
        for i, t in enumerate(suggestions, 1):
            display(Markdown(f"{i}. {t['title']} ({t['type']}) - {t['justification']}"))
        
        # Avviamo un ciclo che consente all'utente di selezionare un topic tra quelli suggeriti. 
        # Il ciclo continua finché l'utente non inserisce un numero valido.
        while True:
            try:
                selected_index = int(input("Scegli il numero del topic da usare: ")) - 1
                if 0 <= selected_index < len(suggestions):
                    selected_topic = suggestions[selected_index]["title"]
                    state["selected_topic"] = selected_topic
                    display(Markdown(f"✅ Topic selezionato: {selected_topic}"))
                    break
                else:
                    display(Markdown("⚠️ Selezione non valida. Riprova."))
            except ValueError:
                display(Markdown("⚠️ Inserisci un numero valido."))

        state["suggested_topics"] = suggestions

        # Salviamo anche l'intera lista dei suggerimenti nello stato, per potervi accedere successivamente se necessario.
        save_state_to_disk(state)
        return state
    
    # Se qualcosa va storto (es. errore nel JSON generato dal modello), gestiamo l’eccezione mostrando un messaggio di errore e ritornando un fallback.
    except (json.JSONDecodeError, ValueError) as e:
        print(f"Errore nel parsing dei suggerimenti: {str(e)}")
        display(Markdown("🔴 Errore: problemi nella generazione dei suggerimenti"))

        return {
            "suggested_topics": [{
                "title": "Errore: suggerimenti non disponibili",
                "type": "error",
                "justification": "Prova a riformulare la richiesta"
            }],
        }


#|---------------------------------------FUNZIONE CHE PERMETTERE GENERARE DELLE RISORSE ADEFUATE DA CUI POI CREARE LA BOZZA---------------------------------------|

# Questa funzione serve a classificare automaticamente un argomento (topic) all’interno di una categoria predefinita tramite un modello linguistico (LLM). 
# L’elenco delle categorie è esplicitamente definito all’inizio della funzione e include settori come viaggi, tecnologia, cucina, ecc. 
# Viene poi costruito un prompt in linguaggio naturale che chiede all’LLM di scegliere una sola categoria esattamente come scritta nell’elenco. 
# Il modello restituisce la risposta, che viene verificata: se corrisponde a una delle categorie, viene restituita; altrimenti, la funzione ritorna "generica" come categoria di fallback. 
def classify_topic_with_llm(topic):
    categories = [
        "viaggi", "tecnologia", "cucina", "programmazione", "fai_da_te", 
        "moda", "arte", "musica", "scrittura", "sport", "cinema_e_tv",
        "fotografia", "finanza_personale", "automobili_e_motori"
    ]
    prompt = f"""Dato l'argomento: "{topic}", scegli **solo** una categoria dalla seguente lista:
        {', '.join(f'"{cat}"' for cat in categories)}.
        Rispondi solo con una delle parole esattamente come scritte sopra, senza aggiungere altro testo.
        """
    response = llm.invoke(prompt)
    return response if response in categories else "generica"

# Questa funzione permette di eseguire la ricerca delle fonti necessarie e valide per la creazione di un post tramite Tavily 
def research_topic(state: BlogState):
    if not state.get("selected_topic"):
        display(Markdown("⚠️ Nessun topic selezionato."))
        state["last_research"] = []
        return []
    
    # Costruisce la query di ricerca a partire dal topic scelto
    topic = state["selected_topic"]
    query = f"{topic}. Guida pratica/tutorial dettagliato con istruzioni passo-passo"
    display(Markdown(f"Ricerca in corso: {query}"))

    # Eseguire la ricerca con Tavily (un tool di search esterno). In caso di errore nella connessione al servizio di ricerca stampiamo un errore
    try:
        raw_results = tavily_tool.invoke({
            "query": query,
            "search_depth": "advanced",
            "max_results": 5,
            "include_answer": False
        })
    except Exception:
        
        display(Markdown("🔴 Errore: Connessione al servizio di ricerca fallita."))
        state["last_research"] = []
        return []
    
    if not isinstance(raw_results, list) or not raw_results:
        display(Markdown("⚠️ Nessun risultato trovato."))
        state["last_research"] = []
        return []
    
    # Classificazione automatica del topic per determinare quali fonti sono affidabili
    category = classify_topic_with_llm(topic)

    # Mappa delle fonti affidabili per ogni categoria
    trusted_domains_by_category = {
        "viaggi": ["alpitour.it", "lonelyplanetitalia.it", "turismo.it", "viaggi.corriere.it", "doveviaggi.it", "skyscanner.it", "momondo.it", "tripadvisor.it", "montagna.tv", "mountainblog.it", "planetmountain.com", "trekking.it"],
        "tecnologia": ["wired.it", "hdblog.it", "tomshw.it", "tech.everyeye.it", "smartworld.it", "hwupgrade.it", "techradar.com", "androidworld.it", "youtube.com"],
        "cucina": ["giallozafferano.it", "ricettedellanonna.net", "cucchiaio.it", "cookaround.com", "lacucinaitaliana.it","fattoincasadabenedetta.it", "buttalapasta.it", "youtube.com"],
        "programmazione": ["aranzulla.it", "html.it", "stackoverflow.com", "geeksforgeeks.org", "dev.to", "w3schools.com", "docs.python.org", "developer.mozilla.org", "youtube.com"],
        "fai_da_te": ["bricolage-faidate.it", "ideegreen.it", "nigiara.it", "bricoportale.it", "lavorincasa.it","faidatemania.pianetadonna.it", "casafacile.it", "youtube.com", "giardinaggio.it"],
        "moda": ["vogue.it", "gqitalia.it", "elle.com/it", "vanityfair.it", "moda.corriere.it", "cosmopolitan.com/it", "youtube.com"],
        "arte": ["domestika.org", "artribune.com", "arte.it", "finestresullarte.info", "exibart.com", "artspecialday.com", "youtube.com"],
        "musica": ["rockol.it", "rollingstone.it", "ondarock.it", "allmusic.com", "metalitalia.com", "billboard.it" "youtube.com"],
        "scrittura": ["narrativamente.com", "thebookishbox.it", "pennablu.it", "scrittorincorso.it", "meloleggo.it", "manualediscrittura.it", "librierecensioni.com", "bookblister.com"],
        "sport": ["gazzetta.it", "corrieredellosport.it", "sportmediaset.it", "tuttosport.com", "figc.it", "calcio.com", "motogp.com", "espn.com", "youtube.com"],
        "cinema_e_tv": ["cineblog.it", "film.it", "mymovies.it", "comingsoon.it", "sky.it/cinema", "netflix.com", "youtube.com"],
        "fotografia": ["fotografareinitalia.it", "milanofotografia.it", "canon.it", "nikon.it", "pixaBay.com", "unsplash.com", "flickr.com"],
        "finanza_personale": ["finanzautile.it", "money.it", "ilsole24ore.com", "lavoce.info", "investireoggi.it", "wallstreetitalia.com"],
        "automobili_e_motori": ["automoto.it", "quattroruote.it", "motori.it", "autobest.co", "caradvice.com", "autoblog.com", "topgear.com"],
        "generica": ["wikipedia.org", "youtube.com", "ilpost.it", "repubblica.it", "corriere.it", "fanpage.it", "ansa.it"]
    }

    # Otteniamo la lista dei domini fidati per la categoria del topic; se non esiste una corrispondenza, usiamo quelli generici
    trusted_domains = trusted_domains_by_category.get(category, trusted_domains_by_category["generica"])

    # Questa parte di codice estrae il dominio dal link, acquisisce contenuto e la data di pubblicazione. 
    # Definisce dei criteri di valutazione per il risultato (domini corretti, lunghezza del contenuto e data di pubblicazione)
    # Calcola un punteggio di affidabilità in base ai criteri sopra
    # Se ha almeno un livello accettabile, lo aggiungiamo tra i risultati valutati
    evaluated_results = []
    for result in raw_results:
        try:
            domain = result.get("url", "").split("//")[-1].split("/")[0].lower()
            content = result.get("content", "")
            date = result.get("published_date", "")
            
            is_trusted = any(dom in domain for dom in trusted_domains)
            is_informative = len(content) > 100
            is_recent = any(year in date for year in ["2022", "2023", "2024", "2025"])
            
            score = sum([is_trusted, is_informative, is_recent])
            if score == 3:
                level = "alta"
            elif score == 2:
                level = "media"
            elif score == 1:
                level = "bassa"
            else:
                level = None

            if level:
                evaluated_results.append({
                    "title": result.get("title", "N/A"),
                    "url": result.get("url", ""),
                    "domain": domain,
                    "level": level,
                    "content": content,
                    "excerpt": content[:200] + "..."
                })
        except Exception:
            continue

    # Ordina i risultati per livello di affidabilità (dalla più alta alla più bassa) e prende solo i primi 3 risultati più affidabili
    evaluated_results.sort(key=lambda x: ["alta", "media", "bassa"].index(x["level"]))
    final_results = evaluated_results[:3]

    # Aggiorna lo stato con i risultati trovati
    state["last_research"] = final_results

    if final_results:
        display(Markdown("🟢 Fonti selezionate:"))
        for res in final_results:
            display(Markdown(f"- [{res['title']}]({res['url']})  \n`{res['domain']}` | Affidabilità: **{res['level']}**"))
    else:
        display(Markdown("⚠️ Nessun risultato valido"))

    return final_results


#|---------------------------------------FUNZIONI CHE PERMETTONO DI APPORTARE DELLE MODIFICHE ALLA BOZZA CREATA---------------------------------------|

# La funzione modify_text(draft) permette di modificare una bozza di testo in un’interfaccia grafica interattiva utilizzando Gradio. 
# Inizia creando un dizionario modified_text per memorizzare la bozza modificata e un oggetto threading.
# Event() per sincronizzare l’aggiornamento del testo. 
# All’interno della funzione annidata modify(draft_modificato), il testo aggiornato viene salvato nel dizionario e viene attivato l’evento per segnalare che la modifica è completata. 
# Successivamente, viene creata un’interfaccia Gradio (gr.Blocks()) con una casella di testo per modificare la bozza e un pulsante di salvataggio. 
# Quando l’utente preme il pulsante, la funzione modify aggiorna il testo e disabilita sia il campo di input che il pulsante, indicando che la bozza è stata salvata. 
# Infine, l’interfaccia Gradio viene eseguita in un thread separato, e la funzione attende (done.wait()) fino a quando la modifica non è stata confermata.
def modify_text(draft):
    modified_text = {"value": draft}
    done = threading.Event()

    def modify(draft_modificato):
        modified_text["value"] = draft_modificato
        done.set()
        return gr.update(value=draft_modificato, interactive=False), gr.update(value="Salvato", interactive=False)
    
    with gr.Blocks() as demo:
        textbox = gr.Textbox(label="Modifica il contenuto della bozza", value=draft, lines=40)
        submit_btn = gr.Button("Salva bozza", variant="primary")

        submit_btn.click(
            fn=modify,
            inputs=[textbox],
            outputs=[textbox, submit_btn]
        )
    threading.Thread(target=demo.launch(quiet=True), kwargs={"share": False}).start()
    done.wait()
    return modified_text["value"]

# Funzione ausiliaria per leggere input testuale e validarlo
def ask_input(prompt, valid_options):
    while True:
        user_input = input(prompt).strip().lower()
        if user_input in valid_options:
            return user_input
        else:
            display(Markdown(f"⚠️ Input non valido, scrivi uno tra: {', '.join(valid_options)}"))


# Funzione principale che genera una bozza del post partendo dal topic selezionato e da uno stile scelto
def generate_post(state: BlogState):
    if "selected_topic" not in state or not state["selected_topic"]:
        display(Markdown("⚠️ Nessun topic selezionato. Scegli un topic dai suggerimenti prima di procedere."))
        return

    title = state["selected_topic"]
    display(Markdown(f"📝 Creazione bozza per il topic: {title}"))

    # Viene scelto lo stile di scrittura della bozza 
    # Si definisce una lista di stili di scrittura disponibili
    # Cicla attraverso la lista degli stili di scrittura e li visualizza numerati
    available_moods = ["professionale", "divertente", "motivazionale", "ironico"]
    display(Markdown("Scegli uno stile di scrittura:"))
    for idx, mood in enumerate(available_moods, 1):
        display(Markdown(f"{idx}. {mood}"))
    
    valid_choices = [str(i) for i in range(1, len(available_moods) + 1)]
    choice = ask_input(f"Inserisci il numero dello stile desiderato (1-{len(available_moods)}): ",valid_choices)
    selected_mood = available_moods[int(choice) - 1]
    display(Markdown(f"🟢 Hai scelto lo stile: **{selected_mood}**"))
    display(Markdown("--------------------------------------------------"))
    
    display(Markdown("🔎 Avvio ricerca per le fonti!"))
    research_topic(state)

    # Recupera l'ultima ricerca salvata nello stato, altrimenti assegna una lista vuota
    research = state.get("last_research", [])

    if not research:
        display(Markdown("⚠️ Nessuna fonte disponibile."))
        return
    
    # Crea un testo che elenca le risorse disponibili, formato come una lista puntata con titolo e URL
    resources_text = "\n".join([ f"- {res['title']} ({res['url']})" for res in research])

    if not resources_text.strip():
        display(Markdown("⚠️ Nessuna fonte utile disponibile per il post."))
        return

    # Prompt che guida il modello nel generare un post completo
    # Questo prompt chiede al modello di generare una bozza in italiano per un post “how-to” (guida pratica) su un argomento specifico, utilizzando risorse fornite, 
    # seguendo uno stile di scrittura selezionato (es. divertente, professionale) e rispettando una struttura dettagliata composta da: 
    # - titolo, 
    # - introduzione,
    # - materiali, 
    # - passaggi numerati, 
    # - consigli, 
    # - errori comuni, 
    # - risorse aggiuntive,
    # - conclusione. 
    # Il risultato deve essere testo semplice e ben formattato, senza Markdown né codice, pronto per la revisione o la pubblicazione
    draft_prompt = f"""**Obiettivo**: Scrivi una **bozza completa** in lingua italiana per un post di tipo **"how-to"** (guida pratica) sul seguente argomento: **Topic**: "{title}"
        ---
        **Risorse da utilizzare**: {resources_text}
        ---
        **Stile richiesto**: {selected_mood}
        ---
        ### Linee guida per la scrittura del post:

        1. **Tono e stile**:
        - Usa un linguaggio coerente con lo stile: {selected_mood}.
        - Il tono deve essere pratico, amichevole e chiaro, adatto anche a lettori senza conoscenze pregresse.
        - Coinvolgi il lettore fin dall’inizio con esempi concreti e applicazioni pratiche dove possibile.

        2. **Struttura del contenuto**:
        - **Titolo**: aggiungi il titolo del topic scelto, cioè: {title}.
        - **Introduzione**: spiega brevemente l’utilità della guida, a chi è rivolta e cosa imparerà il lettore.
        - **Materiali o strumenti necessari** (se rilevanti): elenca tutto ciò che serve per seguire i passaggi.
        - **Passaggi dettagliati**:
            - Presenta ogni passaggio in ordine logico.
            - Per ogni step: spiega *cosa fare*, *perché è importante* e *come farlo al meglio*, includendo eventuali suggerimenti o trucchi.
        - **Consigli pratici**: aggiungi suggerimenti per migliorare l’esperienza o ottenere risultati migliori.
        - **Errori comuni da evitare**: segnala eventuali difficoltà o scelte sbagliate frequenti.
        - **Risorse aggiuntive** (opzionali): link utili, strumenti online, riferimenti.
        - **Conclusione**: incoraggia il lettore a provare, condividere la propria esperienza, o fare domande.

        3. **Formato**:
        - Usa **elenchi numerati** per i passaggi.
        - Evidenzia in **grassetto** i concetti chiave o le parole importanti.
        - Mantieni paragrafi brevi e scorrevoli. Esegui una descrizione più dettagliata ove lo ritieni necessario.
        ---
        ### Output atteso:
        - **Non aggiungere ulteriori titoli** come: "Ecco una bozza del post", "Nuova bozza dal titolo", ecc. Limitati a seguire la struttra fornita.
        - Fornisci il contenuto del post come **testo semplice in lingua italiana**, ben formattato e pronto per essere rivisto o pubblicato. 
        - **Non usare Markdown, codice, o formati strutturati come JSON.**
        """
    
    response = llm.invoke(draft_prompt)
    # Salva la risposta del modello come bozza
    draft = response.content

    display(Markdown("✍️ Ecco una bozza del post:"))
    display(Markdown(draft))

    # Questo blocco di codice gestisce la fase di miglioramento di una bozza appena creata all’interno del sistema. 
    # L’utente viene innanzitutto invitato a scegliere se desidera migliorare la bozza; 
    # Se risponde “sì”, può optare tra due modalità: modifica manuale oppure revisione automatica con feedback. 
    # Se sceglie di modificarla manualmente, si apre un’interfaccia testuale (tramite modify_text) e, se confermata, la nuova versione viene salvata insieme al suo titolo, estratto automaticamente dalla bozza stessa. 
    # Se invece l’utente fornisce un feedback scritto, il sistema genera automaticamente una versione revisionata della bozza, migliorandone contenuto, chiarezza e struttura, senza alterarne stile e tono. 
    # Dopo aver visualizzato la versione revisionata, l’utente può decidere se salvarla oppure mantenere la versione originale. 
    # In ogni caso, alla fine del processo, la bozza (originale o migliorata) viene aggiunta allo stato persistente del sistema, insieme al relativo titolo. 
    # Il topic selezionato viene azzerato per permettere la creazione di un nuovo post.
    user_input = ask_input("Vuoi migliorare la bozza appena creata? (si/no): ", ["si", "no"])
    if user_input == "si":
        user_input_choice = ask_input("Vuoi aggiungere un feedback o modificare la bozza? Scegli (f/m): ", ["f", "m"])
        if user_input_choice == "m":
            updated_draft = modify_text(draft)
            user_confirm = ask_input("Vuoi memorizzare la bozza modificata? (si/No): ", ["si", "no"])
            if user_confirm == "si":
                state["draft"].append(updated_draft)

                match = re.search(r"\*?\*?\s*Titolo\s*\*?\*?:\s*(.+?)(?:\n{2,}|$)", updated_draft, flags=re.IGNORECASE | re.DOTALL)
                if match:
                    extract_title = match.group(1).strip()
                else:
                    extract_title = updated_draft.strip().split("\n", 1)[0].strip()
                state.setdefault("draft_titles", []).append(extract_title)
                
                state["selected_topic"] = None
                save_state_to_disk(state)
                display(Markdown("✅ Bozza modificata salvata."))
            else:
                state["draft"].append(draft)
                state.setdefault("draft_titles", []).append(title)
                state["selected_topic"] = None
                save_state_to_disk(state)
                display(Markdown("✅ Bozza originale salvata."))
        else:
            feedback = input("inserisci il tuo feedback: ").strip().lower()
            if not feedback:
                state["draft"].append(draft)
                state.setdefault("draft_titles", []).append(title)
                save_state_to_disk(state)
                display(Markdown("✅ Feedback non fornito, Bozza originale salvata."))
            else:
                display(Markdown("--------------------------------------------------"))
                display(Markdown(f"🟢 Hai scelto di revisione la bozza sulla base di questo feedback: {feedback}. Fase di revisione avviata.."))

                revision_prompt = f"""**Obiettivo**: Revisiona e migliora il contenuto della bozza di tipo **"how-to"** (guida pratica) tenendo conto del feedback fornito. 
                    **Lo stile, la struttura e la formattazione devono rimanere identici alla bozza originale.** 
                    ---
                    ### **Bozza originale**: {draft}
                    ---
                    ### **Feedback ricevuto**:{feedback}
                    ---
                    ### **Istruzioni per la revisione**:
                    1. **Migliora il contenuto testuale** mantenendo la struttura originale:
                    - Approfondisci i punti segnalati nel {feedback}, fornendo spiegazioni più chiare o dettagliate e pratiche.
                    - Assicurati che i passaggi siano logici, ben spiegati e facilmente comprensibili anche per principianti.
                    - Integra esempi pratici, consigli utili o note aggiuntive dove opportuno.
                    - Mantieni la **coerenza con la bozza originale fornita sopra**: migliora senza stravolgere, a meno che il **feedback lo richieda espressamente**.

                    2. **Mantieni stile e tono**:
                    - Rispetta lo stile e il tono originale richiesto: **{selected_mood}**.

                    3. **Mantieni la struttura della bozza originale**:
                    - Integra le modifiche apportate al contenuto rispettando la seguente struttura:
                        - **'Titolo:'**
                        - **'Introduzione:'**
                        - **'Materiali o strumenti necessari:'**
                        - **'Passaggi dettagliati:'**
                        - **'Consigli pratici:'**
                        - **'Errori comuni da evitare:'**
                        - **'Risorse aggiuntive** (opzionali):'
                        - **'Conclusione:'**
                    - **Non aggiungere ulteriori titoli** come: "Revisione del post", "Nuova bozza revisionata", ecc. Limitati ad inserire il Titolo e a seguire la struttra fornita.

                    4. **Formato**:
                    - Se lo ritini opportuno usa **elenchi numerati** per i passaggi.
                    - Evidenzia in **grassetto** i concetti chiave o le parole importanti.

                    5. **Revisione linguistica e formattazione**:
                    - Correggi eventuali errori grammaticali, sintattici o stilistici.
                    ---
                    ### **Output atteso**:
                    - Fornisci il contenuto del post come **testo semplice in lingua italiana**, ben formattato e pronto per essere rivisto o pubblicato. 
                    - **Non usare Markdown, codice, o formati strutturati come JSON.**
                    """
                
                response = llm.invoke(revision_prompt)
                updated_draft = response.content
                display(Markdown("✍️ Bozza Revisionata:"))
                display(Markdown(updated_draft))
                
                user_input_confirm = ask_input("Vuoi memorizzare la bozza revisionata? (si/no): ", ["si", "no"])
                if user_input_confirm == "si":
                    state["draft"].append(updated_draft)

                    match = re.search(r"\*?\*?\s*Titolo\s*\*?\*?:\s*(.+?)(?:\n{2,}|$)", updated_draft, flags=re.IGNORECASE | re.DOTALL)
                    if match:
                        extract_title = match.group(1).strip()
                    else:
                        extract_title = updated_draft.strip().split("\n", 1)[0].strip()
                    state.setdefault("draft_titles", []).append(extract_title)

                    state["selected_topic"] = None
                    save_state_to_disk(state)
                    display(Markdown("✅ Bozza revisionata salvata."))
                else:
                    state["draft"].append(draft)
                    state.setdefault("draft_titles", []).append(title)
                    state["selected_topic"] = None
                    save_state_to_disk(state)
                    display(Markdown("✅ Bozza originale salvata."))

    else:
        state["draft"].append(draft)
        display(Markdown("✅ Bozza originale salvata."))
        state.setdefault("draft_titles", []).append(title)
        state["selected_topic"] = None
        save_state_to_disk(state)
        display(Markdown("🔁 Seleziona ora un nuovo topic per generare un altro post."))


#|---------------------------------------FUNZIONE CHE PERMETTE DI MOSTARE LE BOZZE SALVATE---------------------------------------|

def show_state(state: BlogState):

    display(Markdown("🟢 Stato attuale del BlogAgent"))
    drafts = state.get("draft", [])

    if drafts:
        display(Markdown("✍️ Bozze salvate:"))
        # Itera su tutte le bozze salvate, mostra l'indice della bozza e il suo contenuto
        for idx, draft in enumerate(drafts, 1):
            display(Markdown(f"🔹 **Bozza {idx}**"))
            display(Markdown(draft))
            display(Markdown("-----------------------------------------------"))

        # Questo blocco di codice permette all’utente di eliminare una bozza precedentemente salvata. 
        # Viene prima generata una lista di opzioni valide (i numeri da 1 al numero totale di bozze, più l’opzione "no" per annullare). 
        # Se l’utente sceglie un numero valido, la bozza corrispondente viene rimossa dalla lista drafts. 
        # Il sistema cerca poi di identificare il titolo della bozza rimossa (usando una serie di espressioni regolari) per eliminarlo anche dalla lista dei titoli state["draft_titles"]. 
        # Inoltre, viene aggiornata la sezione schedule_post rimuovendo eventuali post programmati associati a quel titolo. 
        # Infine, lo stato viene salvato su disco e viene mostrato un messaggio di conferma o, se l’utente ha scelto “no”, un messaggio che segnala che nessuna eliminazione è avvenuta.
        valid_options = [str(i) for i in range(1, len(drafts) + 1)] + ["no"]
        user_choice = ask_input( f"Inserisci il numero della bozza da eliminare oppure 'no' per saltare: ", valid_options)
        if user_choice != "no":
            idx = int(user_choice)
            removed = drafts.pop(idx - 1)

            match = re.search(r"\*?\*?\s*Titolo\s*\*?\*?:\s*(.+?)(?:\n{2,}|$)", removed, flags=re.IGNORECASE | re.DOTALL)
            if not match:
                match = re.search(r"\*\*(.*?)\*\*", removed)
                    
            if match:
                removed_title = match.group(1).strip()
            else:
                removed_title = removed.strip().split("\n", 1)[0].strip()

            if removed_title in state["draft_titles"]:
                state["draft_titles"].remove(removed_title)

            filtered_schedule = []
            for entry in state.get("schedule_post", []):
                if not re.fullmatch(rf"{re.escape(removed_title)}", entry["title"].strip(), flags=re.IGNORECASE):
                    filtered_schedule.append(entry)

            state["schedule_post"] = filtered_schedule
            
            save_state_to_disk(state)

            display(Markdown(f"🗑️ Bozza {idx} eliminata."))
        else:
            display(Markdown("✅ Nessuna bozza eliminata."))
    else:
        display(Markdown("📝 Nessuna post generata. Seleziona prima il topic dai suggerimenti!"))


#|---------------------------------------FUNZIONE CHE PERMETTE DI MOSTARE LE BOZZE PROGRAMMATE---------------------------------------|

def publication_calendar(state: BlogState):

    display(Markdown("📆 Calendario di pubblicazione dei post"))
    if not state.get("draft_titles"):
        display(Markdown("⚠️ Nessuna bozza salvata disponibile per pianificare il calendario."))
        return state

    # Ottiene la data corrente nel formato "giorno/mese/anno" e converte la data corrente in un oggetto `date`
    today_str = datetime.now().strftime("%d/%m/%Y")
    today = datetime.strptime(today_str, "%d/%m/%Y").date()
    # Estrae una copia della lista di post programmati (schedule_post) salvata nello stato globale state, oppure una lista vuota se non esiste ancora.
    calendario = state.get("schedule_post", []).copy() 

    # Estrae i titoli degli eventi già programmati
    sched_title = {entry["title"] for entry in state["schedule_post"]}

    # Il ciclo esamina ciascun titolo nelle bozze salvate e verifica se è già stato programmato. 
    # Se non lo è, viene creato un “prompt” per il modello di linguaggio, che deve suggerire la miglior data di pubblicazione in base a criteri stagionali e di contenuto. 
    # La risposta del modello viene poi estratta come la data di pubblicazione suggerita.
    for titolo in state["draft_titles"]:
        if titolo in sched_title:
            continue

        prompt = f"""Agisci come se fossi un esperto di content marketing e content strategist.
            Il tuo compito è decidere la **migliore data di pubblicazione** per un post di blog con il seguente titolo: "{titolo}"

            ### **Criteri da considerare**:
            - **Contenuto** del post: valuta se è legato a particolari stagioni, eventi o periodi dell’anno.
            - **Stagionalità e rilevanza**: adatta la data in base al contesto stagionale (es. maglioni in autunno/inverno, attività all’aperto in primavera/estate).
            - **Distribuzione realistica**: evita di concentrare tutte le pubblicazioni in pochi giorni.

            ### **Requisiti**:
            - La data **non deve essere antecedente a oggi ({today_str})**.
            - Restituisci **solo una singola data futura**, in **formato `DD/MM/YYYY`** (giorno/mese/anno).
            - Non aggiungere spiegazioni o testo extra: solo la data.

            ### **Esempi di errori da evitare**:
            - Proporre una data nel passato.
            - Scegliere una data incoerente col tipo di contenuto (es. maglioni in piena estate).
            """
        
        response = llm.invoke(prompt)
        pubblication_date = response.content.strip()

        # Successivamente, la bozza viene aggiunta al calendario con la data di pubblicazione suggerita.
        calendario.append({
            "date": pubblication_date,
            "title": titolo,
            "status": "schedulato"
        })

    state["schedule_post"] = calendario

    # ordina i post per data
    state["schedule_post"].sort(key=lambda x: datetime.strptime(x["date"], "%d/%m/%Y").date())

    # Dopo aver pianificato le date di pubblicazione, la funzione aggiorna lo stato di ciascun evento nel calendario, cambiando lo stato da “schedulato” a “pubblicato” 
    # se la data di pubblicazione è uguale o precedente alla data odierna.
    for entry in state["schedule_post"]:
        try:
            scheduled_date = datetime.strptime(entry["date"], "%d/%m/%Y").date()
            if scheduled_date <= today and entry["status"] == "schedulato":
                entry["status"] = "pubblicato"
        except Exception as e:
            print(f"Errore nella data per il post '{entry['title']}': {e}")

    for idx, entry in enumerate(state["schedule_post"], 1):
        status_icon = "✅" if entry["status"] == "pubblicato" else "🕒"
        display(Markdown(f"{idx} - `{entry['date']}` → **{entry['title']}** | stato: {entry['status']} {status_icon}"))

    # Questa porzione di codice mostra all’utente la data di pubblicazione suggerita per un post e gli chiede se desidera modificarla. 
    # Se l’utente risponde “sì”, viene richiesto di inserire l'indice del post per recuperarlo e una nuova data nel formato GG/MM/AAAA. 
    # Il codice controlla che la nuova data sia valida e non sia nel passato: 
    # - se è valida e futura, viene accettata;
    # - altrimenti, viene mantenuta la data suggerita inizialmente.
    # Se l’utente non vuole modificare nulla, la data suggerita viene usata direttamente.
    #scheduled = [entry for entry in calendario if entry["status"] == "schedulato"]

    scheduled_map = {
        str(i + 1): entry
        for i, entry in enumerate(calendario)
        if entry["status"] == "schedulato"
    }

    if scheduled_map:
        do_edit = ask_input("Vuoi modificare la data di un post schedulato? (si/no): ", ["si", "no"])
        if do_edit == "si":
            valid_indexes = list(scheduled_map.keys()) + ["no"]
            idx_choice_str = ask_input("Inserisci il numero del post schedulato da modificare: ", valid_indexes)
            
            entry = scheduled_map[idx_choice_str]
            new_date = input("Inserisci la nuova data nel formato GG/MM/AAAA: ")
            try:
                nuova_data_obj = datetime.strptime(new_date, "%d/%m/%Y").date()
                if nuova_data_obj < today:
                    display(Markdown("⚠️ La data è nel passato. Nessuna modifica eseguita. Mantengo la data precedente."))
                    display(Markdown("-----------------------------------------------"))
                else:
                    entry["date"] = new_date
                    entry["status"] = "schedulato"
                    display(Markdown(f"🔄 Data modificata: il post **{entry['title']}** tornerà nello stato **schedulato** con nuova data `{new_date}`"))
                    display(Markdown("-----------------------------------------------"))
            except ValueError:
                display(Markdown("⚠️ Formato data non valido. Nessuna modifica eseguita. Mantengo la data precedente."))
                display(Markdown("-----------------------------------------------"))
        else:
            display(Markdown("✅ Nessuna modifica eseguita."))
            display(Markdown("-----------------------------------------------"))

    save_state_to_disk(state)
    return state

PARTE 2: menù che permetterà l'avvio di tutto e l'interazione del codice con l'utente

In [ ]:
# La funzione blogManager() è il punto d’ingresso principale per l’interazione con il sistema BlogAgent. 
# Dopo aver caricato lo stato corrente del blog da disco, avvia un ciclo continuo che mostra all’utente un menu testuale con diverse opzioni operative. 
# L’utente può scegliere di:
# 1: generare suggerimenti per nuovi post inserendo un argomento, 
# 2: creare un nuovo post basato su suggerimenti già presenti, 
# 3: visualizzare lo stato attuale (inclusi bozze e programmazioni), 
# 4: pianificare un calendario automatico di pubblicazione per le bozze esistenti. 
# Ogni opzione invoca una funzione dedicata che aggiorna lo stato del sistema, il quale viene poi salvato su disco. 
# Il ciclo si interrompe solo quando l’utente seleziona l’opzione per uscire. 
# La funzione include anche una gestione base degli errori per informare l’utente di eventuali problemi durante l’esecuzione.
def blogManager():
    state = load_state_from_disk()

    while True:
        display(Markdown("|---------- Benvenuto nel BlogAgent ----------|"))
        display(Markdown("1. Genera suggerimenti\n2. Crea post\n3. Mostra stato\n4. Pianifica il calendario\n5. Esci"))
        display(Markdown("|---------------------------------------------|"))
        choice = input("Seleziona una opzione: ").strip()

        if choice == "5":
            break
        try:
            if choice == "1":
                user_input = input("\nInserisci l'argomento della guida pratica: ").strip()
                state = suggest_topics(state, user_input)
            elif choice == "2":
                generate_post(state)
            elif choice == "3":
                show_state(state)
            elif choice == "4":
                state = publication_calendar(state)
            else:
                display(Markdown("⚠️ Scelta non valida"))
        except Exception as e:
            display(Markdown(f"🔴 Errore: {str(e)}"))

if __name__ == "__main__":
    blogManager()

PARTE 3: worflow coerente con l'esecuzione dell'agente

In [ ]:
workflow = StateGraph(BlogState)

#---DEFINIZIONE DEI NODI
workflow.add_node("Caricamento dati", load_state_from_disk)
workflow.add_node("Menù", blogManager)
workflow.add_node("Genera suggerimenti", suggest_topics)
workflow.add_node("Seleziona topic", lambda state: (state, []))
workflow.add_node("Creazione bozza", generate_post)
workflow.add_node("Scelta stile", lambda state: (state, []))
workflow.add_node("Ricerca fonti", generate_post)
workflow.add_node("Bozza", lambda state:(state, []))
workflow.add_node("Review", lambda state: (state, []))
workflow.add_node("Modifica post", modify_text)
workflow.add_node("Feedback post", lambda state: (state, []))
workflow.add_node("Stato dell'agente", show_state)
workflow.add_node("Elimina bozza", lambda state: (state, []))
workflow.add_node("Mostra calendario", publication_calendar)
workflow.add_node("Modifica data", lambda state: (state, []))
workflow.add_node("Esci", lambda state: (state, []))

#---GRAFO DEL MENU' E INIZIO  
workflow.add_edge(START, "Caricamento dati")

workflow.add_edge("Caricamento dati", "Menù")
workflow.add_conditional_edges(
    "Menù",
    lambda state : "1",
    {
        "1": "Genera suggerimenti", 
        "2": "Creazione bozza",  
        "3": "Stato dell'agente",  
        "4": "Mostra calendario", 
        "5": "Esci",
        "errore": "Menù" 
    }
)

#---SCELTA DEI SUGGERIMENTI E SELEZIONE TOPIC
workflow.add_conditional_edges(
    "Genera suggerimenti",
    lambda state: "Seleziona topic" if state.get("suggested_topics") else "Menù",
    {
        "topic": "Seleziona topic", 
        "errore": "Menù"
    }
)
workflow.add_edge("Seleziona topic", "Menù")

#---CREAZIONE E MODIFICA DEL POST
workflow.add_conditional_edges(
    "Creazione bozza",
    lambda state: "Scelta stile" if state.get("selected_topic") else "Menù",
    {
        "Scelta stile": "Scelta stile", 
        "no topic": "Menù"
    }
)
workflow.add_edge("Scelta stile", "Ricerca fonti")
workflow.add_edge("Ricerca fonti", "Bozza")
workflow.add_conditional_edges(
    "Bozza",
    lambda state: "1",
    {
        "no modifica": "Menù",
        "si modifica": "Review"
    }
)
workflow.add_conditional_edges(
    "Review",
    lambda state: "1",
    {
        "m": "Modifica post",
        "f": "Feedback post"
    }
)
workflow.add_edge("Modifica post", "Menù")
workflow.add_edge("Feedback post", "Menù")

#---MOSTRA LO STATO DELL'AGENTE CONTENENTE LE BOZZE 
workflow.add_conditional_edges(
    "Stato dell'agente",
    lambda state: "1",
    {
        "no": "Menù",
        "num_bozza": "Elimina bozza"
    }
)
workflow.add_edge("Elimina bozza", "Menù")

#---MOSTRA IL CALENDARIO CONTENENTE LA PROGRAMMAZIONE DELLE BOZZE
workflow.add_conditional_edges(
    "Mostra calendario",
    lambda state: "1",
    {
        "si": "Modifica data",
        "no": "Menù"
    }
)
workflow.add_edge("Modifica data", "Menù")

workflow.add_edge("Esci", END)

blog_agent = workflow.compile()
display(Image(blog_agent.get_graph().draw_mermaid_png()))